**About** : This notebook is used to infer models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
torch.cuda.get_device_name(0)

In [ ]:
import os
import re
import cv2
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from utils.logger import Config, upload_to_kaggle

from params import *
from data.dataset import SignDataset
from data.preparation import *

from model_zoo.models import define_model
from utils.metrics import *
from utils.torch import load_model_weights
from utils.plots import plot_sample
# from utils.plots import plot_confusion_matrix

from inference.predict import predict, predict_tta
# from inference.main import *

In [ ]:
df = prepare_data(DATA_PATH, "")

### Preprocessing

In [ ]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    df = pd.read_parquet(pq_path)
    n_frames = int(len(df) / ROWS_PER_FRAME)
    data = df[['x', 'y', 'z']].values.reshape(n_frames, ROWS_PER_FRAME, 3)
    return df, data.astype(np.float32)

In [ ]:
KEPT_LANDMARKS = [
    [489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521],
    [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488],
    [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542],
]
MAPPING = [1, 2, 3]  # pose, lhand, rhand

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Preprocessing(nn.Module):
    def __init__(self, type_embed, max_len=50):
        super(Preprocessing, self).__init__()

        self.type_embed = torch.from_numpy(type_embed[None, :].astype(np.float32))
        self.type_embed = self.type_embed.repeat(1000, 1)

        self.landmark_embed = torch.tensor(np.arange(120)).float().unsqueeze(0) + 1
        self.landmark_embed = self.landmark_embed.repeat(1000, 1)
        
        self.ids = torch.from_numpy(np.concatenate(KEPT_LANDMARKS))

        self.to_avg = []

        self.hands = torch.tensor(
            [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488] + 
            [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
        )
        
        self.frames = torch.tensor(np.arange(1000) + 1)
        
        self.max_len = torch.tensor([max_len])
        
        self.shoulders = torch.where((self.ids == 500) | (self.ids == 501))[0]  # torch.tensor([62, 63])  # [500, 501]
        

    def filter_sign(self, x):
        hands = x[:, self.hands, 0]
        nan_prop = torch.isnan(hands).float().mean(-1)            
        x = x[torch.where(nan_prop < 1)[0]]
        
        length = self.frames[:x.size(0)].max().unsqueeze(0)
        sz = torch.cat([length, self.max_len]).max()
        
        divisor = (((sz - self.max_len) > 0) * (sz / self.max_len) + 1).int()
#         if sz > 50:
#             print(sz, divisor)

        ids = (self.frames[:x.size(0)] % divisor) == 0
        
        return x[ids]
        
    def normalize_shoulders(self, x):
        shoulders = x.transpose(0, 1)[self.shoulders]  # 2 x n_frames x 3
        center = ((shoulders[0] + shoulders[1]) / 2).mean(0)
        dist = torch.sqrt(((shoulders[0] - shoulders[1]) ** 2).sum(-1)).mean()
#         print(center, dist)
        return (x - center) / (dist + 1e-6)

    def forward(self, x):
#         x = self.filter_sign(x)
        n_frames = x.shape[0]     
        
        avg_ids = []
        for ids in self.to_avg:
            avg_id = x[:, ids].mean(1, keepdims=True)
            avg_ids.append(avg_id)

        x = torch.cat([x[:, self.ids]] + avg_ids, 1)

        type_embed = self.type_embed[:n_frames]
        landmark_embed = self.landmark_embed[:n_frames, :x.shape[1]]
        
        # Normalize & fill nans
#         nonan = x[~torch.isnan(x)].view(-1, x.shape[-1])
#         x = x - nonan.mean(0)[None, None, :]
#         x = x / nonan.std(0, unbiased=False)[None, None, :]
        
        x[torch.isnan(x)] = 0
        
#         x = self.normalize_shoulders(x)

        # Concat
        x = torch.cat([
            type_embed.unsqueeze(-1), x # , landmark_embed.unsqueeze(-1)
        ], -1).transpose(1, 2)

#         # SIMPLIFIED
#         x = x[:, :, :2].transpose(0, 1)
#         x = x[[ 0, 2, 5, 11, 12, 13, 14, 33, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 58, 59, 62, 63, 66, 67, 70, 71, 74]]
#         x = x.transpose(0, 1).transpose(1, 2)

# #         Resize
#         x = x.contiguous().view(x.size(0), -1).transpose(0, 1)
#         x = F.interpolate(x.unsqueeze(1), 50).squeeze(1)
#         x = x.transpose(0, 1).contiguous().view(50, 5, -1)
        
        # Crop
#         x = x[:self.max_len]
        
        return x

In [ ]:
landmarks = np.concatenate(KEPT_LANDMARKS)
type_embed = np.zeros(1000)
start = 0
for subset, idx in zip(KEPT_LANDMARKS, MAPPING):
    print(subset, idx)
    type_embed[start: start + len(subset)] = idx
    start += len(subset)

type_embed = type_embed[type_embed > 0]

In [ ]:
dataset = SignDataset(df, max_len=None, train=False)

In [ ]:
prepro = Preprocessing(type_embed, max_len=40)

In [ ]:
SAVE = False

SAVE_FOLDER = "../input/openhands/"
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
ts = []
lens = []

for i in tqdm(range(len(df['path']))):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"
    
#     if SAVE and os.path.exists(SAVE_FOLDER + name):
#         continue

    pq, data = load_relevant_data_subset(path)
    data = torch.from_numpy(data)
    
    t0 = time.time()
    
    out_torch = prepro(data)
    
    lens.append(out_torch.size(0))

    assert len(out_torch)

    t1 = time.time()
    
    ts.append((t1 - t0) * 1000)

    if SAVE:
        np.save(SAVE_FOLDER + name, out_torch.numpy())

    if not (i % 10000):
        try:
            data = {
                "x": out_torch[:, 1],
                "y": out_torch[:, 2],
                "z": out_torch[:, 3],
                "type": out_torch[:, 0],
            }
        except:
            data = {
                "x": out_torch[:, 0],
                "y": out_torch[:, 1],
    #             "z": out_torch[:, 3],
                "type": torch.ones_like(out_torch[:, 0]),
            }

#         data["type"][:, [11, 12]] = 4
        plot_sample(data, n_frames=4, figsize=(10, 10))
    
    break
#     if not SAVE:
#         if i > 1000:
#             break

print(f'Prepro : {np.mean(ts) :.1f}ms')

In [ ]:
import pickle

path = '../input/wlasl/wlasl_poses_pickle/'
d = pickle.load(open(path + np.random.choice(os.listdir(path)), 'rb'))['keypoints']

# d = np.where(d[:, :, :1] == 0, np.nan, d)
# x = torch.from_numpy(d)
# nonan = x[~torch.isnan(x)].view(-1, x.shape[-1])
# x = x - nonan.mean(0)[None, None, :]
# x = x / nonan.std(0, unbiased=False)[None, None, :]
# x[torch.isnan(x)] = 0

x = torch.from_numpy(d)

w_data = {
    "x": x[:, :, 0],
    "y": x[:, :, 1],
    "z": x[:, :, 2],
    "type" : torch.tensor([1] * 33 + [2] * 21 + [3] * 21).unsqueeze(0).repeat(d.shape[0], 1)
}

w_data["type"][:, [11, 12]] = 4
w_data["type"][:, 50] = 4

plot_sample(w_data, n_frames=4, figsize=(10, 10))

In [ ]:
sns.histplot(lens)
plt.show()

Done ! 